This notebook will help randomly select images from test set, crop and normalize the images and reconstruct images to form the Human evaluation dataset.

* Fidelity evaluation
* Alignment evaluation
    * Build dataset
    * Get reconstructed images from different methods
* Prepare for Qualtrics survey

## Helper functions

In [1]:
import cv2
import os
import re
import numpy as np
import pickle as pkl
import torch
import dnnlib
import legacy
import pandas as pd
import random
import shutil

SAMPLES = 100
ROOT_DIR = "/home/xavier/Documents/dataset/Welch/trainingset2/trainingset2"
CLASS_DIR = "/home/xavier/Documents/dataset/Welch/classification-v2024/selected_images"
SPLIT_DICT = "/home/xavier/Documents/dataset/Welch/classification-v2024/classification_models/240430-001/movie_classification/generator_full_dataset.csv"
model_dict = {
    13: "/home/xavier/PycharmProjects/training-runs/e13-post/from302kimgs/00001-stylegan2-trainingset2-gpus4-batch112-gamma10/network-snapshot-001512.pkl"
}
OUT_DIR = "/media/xavier/Storage/model_evaluation"
os.makedirs(OUT_DIR, exist_ok=True)


def resize_crop(img_dir, resize_by=1., resolution=512, brightness_norm=True, brightness_mean=107, locations=None):
    if locations is None:
        locations = ["left", "right"]
    img = cv2.imread(img_dir, cv2.IMREAD_UNCHANGED)
    if img.dtype != np.uint8:
        img = np.uint8(img / 256)
    # img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    img_shape = img.shape
    resize_shape = np.array([img_shape[1] * resize_by, img_shape[0] * resize_by], dtype=int)
    if resize_by != 1:
        img = cv2.resize(img, resize_shape, cv2.INTER_LANCZOS4)
    imgs = []
    for location in locations:
        if location == "left":
            new_img = img[(resize_shape[1] - resolution) // 2:(resize_shape[1] + resolution) // 2, :resolution]
        elif location == "right":
            new_img = img[(resize_shape[1] - resolution) // 2:(resize_shape[1] + resolution) // 2, -resolution:]
        else:
            new_img = img[(resize_shape[1] - resolution) // 2:(resize_shape[1] + resolution) // 2,
                      (resize_shape[0] - resolution) // 2:(resize_shape[0] + resolution) // 2]
        if brightness_norm:
            obj_v = np.mean(new_img)
            value = brightness_mean - obj_v
            new_img = cv2.add(new_img, value)
        imgs.append(new_img)
    return imgs


## Get the dataset table

In [52]:
container = []
for strain in os.listdir(ROOT_DIR):
    run_id = int(strain.split("Run")[1])
    for scope in os.listdir(os.path.join(ROOT_DIR, strain)):
        container.append([run_id, int(scope[-2:]), strain, scope])

container_df = pd.DataFrame(container, columns=["run_id", "scope_id", "strain", "scope"])
scope_count = container_df.groupby('strain')['scope'].nunique().reset_index()
scope_count.columns = ['strain', 'scope_count']

split_df = pd.read_csv(SPLIT_DICT)

merged_df = pd.merge(container_df, split_df, on=['run_id', 'scope_id'])
merged_df = merged_df.merge(scope_count, on='strain', how='left')
merged_df = merged_df[~(merged_df['Category'] == 'Training')]
merged_df = merged_df.drop(columns=['directory'])
merged_df

,run_id,scope_id,strain,scope,Category,scope_count
2,636,39,CS5_78_0425_1%agar_Run0636,Scope39,Test,3
8,451,18,CS4_04_4399_1%agar_Run0451,Scope18,Validation,3
21,421,40,CS3_51_5247_1%agar_Run0421,Scope40,Validation,3
23,286,36,CS1_61_7827_1%agar_Run0286,Scope36,Test,2
24,737,6,CS7_19_4464_1%agar_Run0737,Scope06,Validation,2
...,...,...,...,...,...,...
908,254,78,CS1_69_7154_1%agar_Run0254,Scope78,Validation,3
914,747,40,CS7_44_7669_1%agar_Run0747,Scope40,Validation,3
919,423,36,CS3_60_4380_1%agar_Run0423,Scope36,Validation,3
929,238,7,CS1_03_1909_1%agar_Run0238,Scope07,Validation,2


# 1. Select alignment evaluation dataset

In [53]:
def select_frame(path, frame_index_threshold=300):
    images = [img for img in os.listdir(path) if img.endswith(('.png', '.jpg', '.jpeg'))]
    valid_images = [img for img in images if int(img[-8:-4]) > frame_index_threshold]
    if not valid_images:
        return None
    return random.choice(valid_images)


def select_frame_with_matching_index(row, frame_index_threshold=300):
    primary_frame = select_frame(f"{ROOT_DIR}/{row['strain']}/{row['scope']}", frame_index_threshold)
    if primary_frame:
        frame_index = primary_frame[-8:-4]
        additional_path = f"{ROOT_DIR}/{row['strain']}/{row['scope_additional']}"
        additional_images = [img for img in os.listdir(additional_path) if img.endswith(('.png', '.jpg', '.jpeg'))]
        additional_frame = next((img for img in additional_images if img[-8:-4] == frame_index), None)
        return primary_frame, additional_frame
    return primary_frame, None


alignment_candidates = merged_df[merged_df['scope_count'] >= 2]
alignment_eval_set = alignment_candidates.drop_duplicates(subset='strain').sample(n=SAMPLES, random_state=1)

# Find another "scope" for each strain in the alignment evaluation set from container_df
additional_scopes = []
for _, row in alignment_eval_set.iterrows():
    other_scopes = container_df[(container_df['strain'] == row['strain']) & (container_df['scope'] != row['scope'])]
    if not other_scopes.empty:
        additional_scopes.append(other_scopes.iloc[0])

additional_scopes_df = pd.DataFrame(additional_scopes)
alignment_eval_set = alignment_eval_set.merge(additional_scopes_df[['strain', 'scope']], on='strain',
                                              suffixes=('', '_additional'))
alignment_eval_set[['selected_frame', 'selected_frame_additional']] = alignment_eval_set.apply(
    lambda row: pd.Series(select_frame_with_matching_index(row)), axis=1
)
alignment_eval_set.to_csv(f"{OUT_DIR}/alignment_eval.csv", index=False)
alignment_eval_set

,run_id,scope_id,strain,scope,Category,scope_count,scope_additional,selected_frame,selected_frame_additional
0,477,5,CS4_29_4141_1%agar_Run0477,Scope05,Validation,3,Scope06,Run0477_scope5-00_0847.jpg,Run0477_scope6-00_0847.jpg
1,721,39,CS7_42_4519_1%agar_Run0721,Scope39,Test,3,Scope27,Run0721_scope39-00_0989.jpg,Run0721_scope27-00_0989.jpg
2,647,24,CS6_19_4530_1%agar_Run0647,Scope24,Test,2,Scope23,Run0647_scope24-00_0791.jpg,Run0647_scope23-00_0791.jpg
3,444,75,CS3_78_0777_1%agar_Run0444,Scope75,Validation,3,Scope78,Run0444_scope75-00_0613.jpg,Run0444_scope78-00_0613.jpg
4,234,64,CS1_70_0741_1%agar_Run0234,Scope64,Validation,2,Scope63,Run0234_scope64-00_0796.jpg,Run0234_scope63-00_0796.jpg
...,...,...,...,...,...,...,...,...,...
95,666,8,CS6_25_4290_1%agar_Run0666,Scope08,Validation,3,Scope07,Run0666_scope8-00_0770.jpg,Run0666_scope7-00_0770.jpg
96,485,1,CS4_38_3186_%agar_Run0485,Scope01,Test,3,Scope03,Run0485_scope1-00_1099.jpg,Run0485_scope3-00_1099.jpg
97,434,14,CS3_69_1224_1%agar_Run0434,Scope14,Validation,3,Scope15,Run0434_scope14-00_0513.jpg,Run0434_scope15-00_0513.jpg
98,493,27,CS4_46_4861_1%agar_Run0493,Scope27,Validation,2,Scope38,Run0493_scope27-00_0768.jpg,Run0493_scope38-00_0768.jpg


# 2. Select fidelity evaluation dataset

In [58]:
remaining_pairs = merged_df.merge(alignment_eval_set[['strain', 'scope']], on=['strain', 'scope'], how='left',
                                  indicator=True)
fidelity_eval_set = remaining_pairs[remaining_pairs['_merge'] == 'left_only'].drop(columns=['_merge'])
fidelity_eval_set['selected_frame'] = fidelity_eval_set.apply(
    lambda row: select_frame(f"{ROOT_DIR}/{row['strain']}/{row['scope']}"), axis=1)
fidelity_eval_set.to_csv(f"{OUT_DIR}/fidelity_eval.csv", index=False)
fidelity_eval_set

,run_id,scope_id,strain,scope,Category,scope_count,selected_frame
1,451,18,CS4_04_4399_1%agar_Run0451,Scope18,Validation,3,Run0451_scope18-00_0749.jpg
3,286,36,CS1_61_7827_1%agar_Run0286,Scope36,Test,2,Run0286_scope36-00_1352.jpg
5,737,4,CS7_19_4464_1%agar_Run0737,Scope04,Test,2,Run0737_scope4-00_1246.jpg
8,300,47,CS2_07_FruATc_1%agar_Run0300,Scope47,Test,3,Run0300_scope47-00_1050.jpg
9,741,16,CS7_57_4481_1%agar_Run0741,Scope16,Validation,3,Run0741_scope16-00_0483.jpg
...,...,...,...,...,...,...,...
189,738,8,CS7_47_7059_1%agar_Run0738,Scope08,Validation,3,Run0738_scope8-00_1398.jpg
191,284,26,CS1_50_1217_1%agar_Run0284,Scope26,Validation,3,Run0284_scope26-00_0303.jpg
195,254,78,CS1_69_7154_1%agar_Run0254,Scope78,Validation,3,Run0254_scope78-00_1230.jpg
196,747,40,CS7_44_7669_1%agar_Run0747,Scope40,Validation,3,Run0747_scope40-00_1316.jpg


## Build datasets

In [59]:
fidelity_eval_set = pd.read_csv(f"{OUT_DIR}/fidelity_eval.csv")
alignment_eval_set = pd.read_csv(f"{OUT_DIR}/alignment_eval.csv")

fidelity_dataset = f"{OUT_DIR}/fidelity/real"
os.makedirs(fidelity_dataset, exist_ok=True)
for idx, row in fidelity_eval_set.iterrows():
    image = resize_crop(f"{ROOT_DIR}/{row['strain']}/{row['scope']}/{row['selected_frame']}", locations='center')[0]
    cv2.imwrite(f"{fidelity_dataset}/{row['selected_frame']}", image)

alignment_dataset = f"{OUT_DIR}/alignment"
os.makedirs(alignment_dataset, exist_ok=True)


def save_image(image, container, filename):
    path = os.path.join(alignment_dataset, container)
    if not os.path.exists(path):
        os.makedirs(path)
    cv2.imwrite(os.path.join(path, filename), image)


imgs_orig = []
imgs_other = []
imgs_rep = []
img_names = []
for idx, row in alignment_eval_set.iterrows():
    left_crop, right_crop = resize_crop(f"{ROOT_DIR}/{row['strain']}/{row['scope']}/{row['selected_frame']}")
    copy_crop = \
        resize_crop(f"{ROOT_DIR}/{row['strain']}/{row['scope_additional']}/{row['selected_frame_additional']}",
                    locations=['left'])[0]
    save_image(left_crop, 'crops_l', f"{row['strain']}_{row['scope']}_left.jpg")
    save_image(right_crop, 'crops_r', f"{row['strain']}_{row['scope']}_right.jpg")
    save_image(copy_crop, 'crops_cp', f"{row['strain']}_{row['scope_additional']}_left.jpg")
    imgs_orig.append(left_crop[np.newaxis, np.newaxis, :, :])
    imgs_other.append(right_crop[np.newaxis, np.newaxis, :, :])
    imgs_rep.append(copy_crop[np.newaxis, np.newaxis, :, :])
    img_names.append(f"{row['strain']}_{row['scope']}.jpg")

# 3. Get our model generated and reconstructed images
Run gen_images.py.

In [33]:
import os
import re
from typing import List, Optional, Tuple, Union

import click
import dnnlib
import numpy as np
import PIL.Image
import torch

import legacy

os.environ['CC'] = "/usr/bin/gcc-9"
os.environ['CXX'] = "/usr/bin/g++-9"


def parse_range(s: Union[str, List]) -> List[int]:
    if isinstance(s, list): return s
    ranges = []
    range_re = re.compile(r'^(\d+)-(\d+)$')
    for p in s.split(','):
        m = range_re.match(p)
        if m:
            ranges.extend(range(int(m.group(1)), int(m.group(2)) + 1))
        else:
            ranges.append(int(p))
    return ranges


def parse_vec2(s: Union[str, Tuple[float, float]]) -> Tuple[float, float]:
    if isinstance(s, tuple): return s
    parts = s.split(',')
    if len(parts) == 2:
        return (float(parts[0]), float(parts[1]))
    raise ValueError(f'cannot parse 2-vector {s}')


def make_transform(translate: Tuple[float, float], angle: float):
    m = np.eye(3)
    s = np.sin(angle / 360.0 * np.pi * 2)
    c = np.cos(angle / 360.0 * np.pi * 2)
    m[0][0] = c
    m[0][1] = s
    m[0][2] = translate[0]
    m[1][0] = -s
    m[1][1] = c
    m[1][2] = translate[1]
    return m


def generate_images(
        network_pkl: str,
        seeds: List[int],
        truncation_psi: float,
        noise_mode: str,
        outdir: str,
        translate: Tuple[float, float],
        rotate: float,
        class_idx: Optional[int]
):
    print('Loading networks from "%s"...' % network_pkl)
    device = torch.device('cuda')
    with dnnlib.util.open_url(network_pkl) as f:
        G = legacy.load_network_pkl(f)['G_ema'].to(device)  # type: ignore

    os.makedirs(outdir, exist_ok=True)

    # Labels.
    label = torch.zeros([1, G.c_dim], device=device)
    if G.c_dim != 0:
        if class_idx is None:
            raise click.ClickException('Must specify class label with --class when using a conditional network')
        label[:, class_idx] = 1
    else:
        if class_idx is not None:
            print('warn: --class=lbl ignored when running on an unconditional network')

    # Generate images.
    for seed_idx, seed in enumerate(seeds):
        print('Generating image for seed %d (%d/%d) ...' % (seed, seed_idx, len(seeds)))
        z = torch.from_numpy(np.random.RandomState(seed).randn(1, G.z_dim)).to(device)

        if hasattr(G.synthesis, 'input'):
            m = make_transform(translate, rotate)
            m = np.linalg.inv(m)
            G.synthesis.input.transform.copy_(torch.from_numpy(m))

        img = G(z, label, truncation_psi=truncation_psi, noise_mode=noise_mode)
        img = (img.permute(0, 2, 3, 1) * 127.5 + 128).clamp(0, 255).to(torch.uint8)
        if img.shape[3] == 1:
            PIL.Image.fromarray(img[0, :, :, 0].cpu().numpy(), 'L').save(f'{outdir}/dim_{model_e}_seed{seed:04d}.jpg')
        else:
            PIL.Image.fromarray(img[0].cpu().numpy(), 'RGB').save(f'{outdir}/dim_{model_e}_seed{seed:04d}.jpg')


for model_e, model_path in model_dict.items():
    generate_images(model_path, list(range(SAMPLES)), 1, 'random',
                    os.path.join(OUT_DIR, 'fidelity', f'dim_{model_e}_generations'), (0, 0), 0, None)

Loading networks from "/home/xavier/PycharmProjects/training-runs/e13-post/from302kimgs/00001-stylegan2-trainingset2-gpus4-batch112-gamma10/network-snapshot-001512.pkl"...
Generating image for seed 0 (0/100) ...
Generating image for seed 1 (1/100) ...
Generating image for seed 2 (2/100) ...
Generating image for seed 3 (3/100) ...
Generating image for seed 4 (4/100) ...
Generating image for seed 5 (5/100) ...
Generating image for seed 6 (6/100) ...
Generating image for seed 7 (7/100) ...
Generating image for seed 8 (8/100) ...
Generating image for seed 9 (9/100) ...
Generating image for seed 10 (10/100) ...
Generating image for seed 11 (11/100) ...
Generating image for seed 12 (12/100) ...
Generating image for seed 13 (13/100) ...
Generating image for seed 14 (14/100) ...
Generating image for seed 15 (15/100) ...
Generating image for seed 16 (16/100) ...
Generating image for seed 17 (17/100) ...
Generating image for seed 18 (18/100) ...
Generating image for seed 19 (19/100) ...
Generati

In [ ]:
os.environ['CC'] = "/usr/bin/gcc-9"
os.environ['CXX'] = "/usr/bin/g++-9"
device = torch.device('cuda')
os.makedirs(OUT_DIR, exist_ok=True)
# Now calculate and add the similarities for each model's reconstructions
for model_e, model_path in model_dict.items():
    with dnnlib.util.open_url(model_path) as fp:
        models = legacy.load_network_pkl(fp)
    E, G = models['E_ema'].to(device), models['G_ema'].to(device)

    similarities = []
    for img_l, img_name in zip(imgs_orig, img_names):
        img_l_tensor = torch.tensor(img_l, device=device, dtype=torch.float32).div(127.5).sub(1)
        mu, logvar = E.mu_var(img_l_tensor, None)
        recon = G(mu, None).detach()
        recon_clipped = torch.clip(recon, -1, 1)
        recon_rescaled = recon_clipped.add(1).div(2).mul(255).type(torch.uint8)
        recon_output = recon_rescaled.detach().cpu().numpy()[0, 0]
        subdir = f'dim_{model_e}_reconstructions'
        save_image(recon_output, subdir, os.path.basename(img_name))

# 4. Get StyleGAN2 generated and reconstructed images
* Train the StyleGAN2 model with the forked StyleGAN3 project.
* Use gen_images.py to generate random images.
> **_Code_**  
> python gen_images.py --network /home/xavier/PycharmProjects/training-runs/stylegan2/00000-stylegan2-trainingset2-gpus4-batch112-gamma10/network-snapshot-001512.pkl --seeds 0-99 --outdir /media/xavier/Storage/model_evaluation/fidelity/stylegan2 --noise-mode random

* Use project_all.sh to get the other reconstruction.

In [2]:
PROJECT_DIR = "/media/xavier/Storage/model_evaluation/alignment/crops_bp"
COLLECT_DIR = "/media/xavier/Storage/model_evaluation/alignment/stylegan2_reconstructions"
os.makedirs(COLLECT_DIR, exist_ok=True)

for folder in os.listdir(PROJECT_DIR):
    shutil.copy(os.path.join(PROJECT_DIR, folder, "proj.png"), os.path.join(COLLECT_DIR, f"{folder}.png"))


# 5. Build the names for Qualtrics survey

After obtaining the images, follow these steps to build a Turing test in Qualtrics:

1. **Upload images:** Upload all images to the Library, preferably organized into separate folders.

2. **Create a project:** Start a new project and use Loop & Merge to generate multiple questions with the same format. Use `$${lm://Field/index}` to indicate the fields that will change in each quiz.

3. **Configure Loop & Merge:** Create a table in Loop & Merge with columns equal to the number of fields and rows equal to the number of quizzes.

4. **Add images:** To insert images, use the HTML format: `<img src="img_url" style="width:256px;height:256px;"/>`. Replace "img_url" with the actual image URL. To find the URL of the uploaded images, follow these steps:

   - Go to Account Settings -> Qualtrics ID.
   - Click on the Library where the images are uploaded, under the Library section.
   - Scroll down to the folder containing the images.
   - Select the graphics name and ID and paste them into an Excel sheet, creating two columns.
   - Use Excel's concatenate formula to generate URLs in this format: `https://YOUR_DATACENTER_ID.qualtrics.com/ControlPanel/Graphic.php?IM=IMAGE_ID`

5. **Add `${lm://CurrentLoopNumber}` in the question stem.** Go to left Survey flow -> Add a New Element Here (on the top) -> Set Embedded Data -> `index = ${lm://CurrentLoopNumber}`

6. **Generate test responses** using Tools -> Generate test responses.


In [7]:
import pandas as pd
import re
import numpy as np

file_path = '/media/xavier/Storage/model_evaluation/qualtrics_table.xlsx'
out_path = '/media/xavier/Storage/model_evaluation/qualtrics_expression.xlsx'

crops_l_df = pd.read_excel(file_path, sheet_name='crops_l')
crops_r_df = pd.read_excel(file_path, sheet_name='crops_r')
crops_cp_df = pd.read_excel(file_path, sheet_name='crops_cp')
stylegan2_reconstructions_df = pd.read_excel(file_path, sheet_name='stylegan2_reconstructions')
dim_13_reconstructions_df = pd.read_excel(file_path, sheet_name='dim_13_reconstructions')
our_df = pd.read_excel(file_path, sheet_name='our')
real_df = pd.read_excel(file_path, sheet_name='real')
stylegan2_df = pd.read_excel(file_path, sheet_name='stylegan2')
stylegan3_df = pd.read_excel(file_path, sheet_name='stylegan3')


# Function to extract run_id and scope_id from img_name
def extract_ids(img_name):
    match = re.search(r'_(Run\d{4})_(Scope\d{2})', img_name)
    if match:
        return match.group(1), match.group(2)
    return None, None


# Add columns for run_id and scope_id
for df in [crops_l_df, crops_r_df, crops_cp_df, stylegan2_reconstructions_df, dim_13_reconstructions_df]:
    df['run_id'], df['scope_id'] = zip(*df['img_name'].map(extract_ids))

crops_l_df.rename(columns={'url': 'url_l', 'img_name': 'img_name_l'}, inplace=True)
crops_r_df.rename(columns={'url': 'url_r', 'img_name': 'img_name_r'}, inplace=True)
crops_cp_df.rename(columns={'url': 'url_cp', 'img_name': 'img_name_cp'}, inplace=True)
stylegan2_reconstructions_df.rename(columns={'url': 'url_stylegan2', 'img_name': 'img_name_stylegan2'}, inplace=True)
dim_13_reconstructions_df.rename(columns={'url': 'url_dim13', 'img_name': 'img_name_dim13'}, inplace=True)

# Merge dataframes on run_id and scope_id
alignment_df = crops_l_df.merge(crops_r_df, on=['run_id', 'scope_id'], suffixes=('_l', '_r'))
alignment_df = alignment_df.merge(crops_cp_df[['img_name_cp', 'run_id', 'url_cp']], on=['run_id'])
alignment_df = alignment_df.merge(stylegan2_reconstructions_df, on=['run_id', 'scope_id'])
alignment_df = alignment_df.merge(dim_13_reconstructions_df, on=['run_id', 'scope_id'])

# Function to generate the image URL
url_function = lambda \
        x: f'<img src="https://riceuniversity.co1.qualtrics.com/ControlPanel/Graphic.php?IM={x}" style="width:256px;height:256px;"/>'

# Apply the URL function to the 'url' columns
alignment_df['image_expression_l'] = alignment_df['url_l'].apply(url_function)
alignment_df['image_expression_r'] = alignment_df['url_r'].apply(url_function)
alignment_df['image_expression_cp'] = alignment_df['url_cp'].apply(url_function)
alignment_df['image_expression_stylegan2'] = alignment_df['url_stylegan2'].apply(url_function)
alignment_df['image_expression_dim13'] = alignment_df['url_dim13'].apply(url_function)

alignment_df = alignment_df[
    ['run_id', 'scope_id', 'img_name_l', 'img_name_r', 'img_name_cp', 'img_name_stylegan2', 'img_name_dim13', 'url_l',
     'url_r', 'url_cp', 'url_stylegan2', 'url_dim13', 'image_expression_l', 'image_expression_r', 'image_expression_cp',
     'image_expression_stylegan2', 'image_expression_dim13']]
alignment_df

,run_id,scope_id,img_name_l,img_name_r,img_name_cp,img_name_stylegan2,img_name_dim13,url_l,url_r,url_cp,url_stylegan2,url_dim13,image_expression_l,image_expression_r,image_expression_cp,image_expression_stylegan2,image_expression_dim13
0,Run0238,Scope07,CS1_03_1909_1%agar_Run0238_Scope07_left.jpg,CS1_03_1909_1%agar_Run0238_Scope07_right.jpg,CS1_03_1909_1%agar_Run0238_Scope08_left.jpg,CS1_03_1909_1%agar_Run0238_Scope07_left.png,CS1_03_1909_1%agar_Run0238_Scope07.jpg,IM_1bQstIafZazgJhD,IM_wZoApkTwMHxczyV,IM_Zw9I0uxx7Tcle9E,IM_jv5ZCzwq5fzcM70,IM_55eBfaRR4xhPwqW,"<img src=""https://riceuniversity.co1.qualtrics...","<img src=""https://riceuniversity.co1.qualtrics...","<img src=""https://riceuniversity.co1.qualtrics...","<img src=""https://riceuniversity.co1.qualtrics...","<img src=""https://riceuniversity.co1.qualtrics..."
1,Run0249,Scope47,CS1_14_4323_1%agar_Run0249_Scope47_left.jpg,CS1_14_4323_1%agar_Run0249_Scope47_right.jpg,CS1_14_4323_1%agar_Run0249_Scope46_left.jpg,CS1_14_4323_1%agar_Run0249_Scope47_left.png,CS1_14_4323_1%agar_Run0249_Scope47.jpg,IM_0Fmj2Epk3rOmQZK,IM_z2SDPgGssYgNNeX,IM_TCcWg5zUpwDWIW7,IM_3fE5FBm6dRDTVe9,IM_TL4ioptvlweApaR,"<img src=""https://riceuniversity.co1.qualtrics...","<img src=""https://riceuniversity.co1.qualtrics...","<img src=""https://riceuniversity.co1.qualtrics...","<img src=""https://riceuniversity.co1.qualtrics...","<img src=""https://riceuniversity.co1.qualtrics..."
2,Run0261,Scope12,CS1_22_1622_1%agar_Run0261_Scope12_left.jpg,CS1_22_1622_1%agar_Run0261_Scope12_right.jpg,CS1_22_1622_1%agar_Run0261_Scope11_left.jpg,CS1_22_1622_1%agar_Run0261_Scope12_left.png,CS1_22_1622_1%agar_Run0261_Scope12.jpg,IM_YPeZ7j7LzkNiSKX,IM_fDpeydtJqLDbyVk,IM_6KzljR7hrJXOkm8,IM_Vnf6cs6bHSmJIEZ,IM_wBLWvRioskrfFW8,"<img src=""https://riceuniversity.co1.qualtrics...","<img src=""https://riceuniversity.co1.qualtrics...","<img src=""https://riceuniversity.co1.qualtrics...","<img src=""https://riceuniversity.co1.qualtrics...","<img src=""https://riceuniversity.co1.qualtrics..."
3,Run0271,Scope31,CS1_33_A1622_1%agar_Run0271_Scope31_left.jpg,CS1_33_A1622_1%agar_Run0271_Scope31_right.jpg,CS1_33_A1622_1%agar_Run0271_Scope21_left.jpg,CS1_33_A1622_1%agar_Run0271_Scope31_left.png,CS1_33_A1622_1%agar_Run0271_Scope31.jpg,IM_4k52GcBpGwsfiuo,IM_bNzV6BaN7OR1JZU,IM_YzWUdwTYRfDPrT1,IM_1kvW308nUkwg4os,IM_UEVwxhfKDkslEFn,"<img src=""https://riceuniversity.co1.qualtrics...","<img src=""https://riceuniversity.co1.qualtrics...","<img src=""https://riceuniversity.co1.qualtrics...","<img src=""https://riceuniversity.co1.qualtrics...","<img src=""https://riceuniversity.co1.qualtrics..."
4,Run0276,Scope23,CS1_39_4506_1%agar_Run0276_Scope23_left.jpg,CS1_39_4506_1%agar_Run0276_Scope23_right.jpg,CS1_39_4506_1%agar_Run0276_Scope22_left.jpg,CS1_39_4506_1%agar_Run0276_Scope23_left.png,CS1_39_4506_1%agar_Run0276_Scope23.jpg,IM_fA3wMxagrLZ5SJC,IM_8jlZiDIfyHzdsNF,IM_GSx93D7bfScNumL,IM_mmv1pLBqHAzAn2F,IM_iInUPMijizW9stI,"<img src=""https://riceuniversity.co1.qualtrics...","<img src=""https://riceuniversity.co1.qualtrics...","<img src=""https://riceuniversity.co1.qualtrics...","<img src=""https://riceuniversity.co1.qualtrics...","<img src=""https://riceuniversity.co1.qualtrics..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,Run0560,Scope39,MX41_6058_1%agar_Run0560_Scope39_left.jpg,MX41_6058_1%agar_Run0560_Scope39_right.jpg,MX41_6058_1%agar_Run0560_Scope40_left.jpg,MX41_6058_1%agar_Run0560_Scope39_left.png,MX41_6058_1%agar_Run0560_Scope39.jpg,IM_cjbREJMP7jZOdJu,IM_8ZqEgrNkmHE4PNs,IM_GbNxtx1ozSreYjY,IM_aX3bnDJHpTWd8BJ,IM_ABZGNA6WnzxN3gv,"<img src=""https://riceuniversity.co1.qualtrics...","<img src=""https://riceuniversity.co1.qualtrics...","<img src=""https://riceuniversity.co1.qualtrics...","<img src=""https://riceuniversity.co1.qualtrics...","<img src=""https://riceuniversity.co1.qualtrics..."
96,Run0552,Scope08,MX48_6759_1%agar_Run0552_Scope08_left.jpg,MX48_6759_1%agar_Run0552_Scope08_right.jpg,MX48_6759_1%agar_Run0552_Sco

In [20]:
our_df['image_expression'] = our_df['url'].apply(url_function)
real_df['image_expression'] = real_df['url'].apply(url_function)
stylegan2_df['image_expression'] = stylegan2_df['url'].apply(url_function)
stylegan3_df['image_expression'] = stylegan3_df['url'].apply(url_function)

final_columns = []

# Each row in the final DataFrame
for row_idx in range(25):
    row_data = []
    for col_group in [our_df, real_df, stylegan2_df]:
        for col_idx in range(4):
            img_name = col_group.iloc[row_idx * 4 + col_idx]['img_name']
            img_url = col_group.iloc[row_idx * 4 + col_idx]['url']
            img_expr = col_group.iloc[row_idx * 4 + col_idx]['image_expression']
            row_data.extend([img_name, img_url, img_expr])

    # Add the stylegan3_df item to each row
    img_name = stylegan3_df.iloc[row_idx]['img_name']
    img_url = stylegan3_df.iloc[row_idx]['url']
    img_expr = stylegan3_df.iloc[row_idx]['image_expression']
    row_data.extend([img_name, img_url, img_expr])

    final_columns.append(row_data)

# Create the final DataFrame
column_labels = []
for i in range(4):
    for group in ['our', 'real', 'stylegan2']:
        column_labels.extend([f'{group}_img_name_{i}', f'{group}_url_{i}', f'{group}_image_expression_{i}'])
column_labels.extend([f'stylegan3_img_name', f'stylegan3_url', f'stylegan3_image_expression'])

fidelity_df = pd.DataFrame(final_columns, columns=column_labels)

# Rearrange columns to move 'image_expression' to the right
expression_columns = [col for col in fidelity_df.columns if 'image_expression' in col]
other_columns = [col for col in fidelity_df.columns if 'image_expression' not in col]
fidelity_df = fidelity_df[other_columns + expression_columns]

expression_table = fidelity_df[expression_columns]

shuffled_expression_table = pd.DataFrame(index=expression_table.index, columns=expression_table.columns)
source_table = pd.DataFrame(index=expression_table.index, columns=[f'Original_Column_{i}' for i in range(13)])

for idx, row in expression_table.iterrows():
    # Shuffle the row and get the new order of indices
    shuffled_indices = np.random.permutation(len(row))
    shuffled_values = row.values[shuffled_indices]

    # Assign the shuffled values to the new DataFrame
    shuffled_expression_table.loc[idx] = shuffled_values

    # Track the original columns for each value in the shuffled row
    original_columns = [row.index[i] for i in shuffled_indices]
    source_table.loc[idx] = original_columns

# Rename the columns in shuffled_expression_table to a standard naming convention
shuffled_expression_table.columns = [f'Column_{i}' for i in range(13)]

source_table

,Original_Column_0,Original_Column_1,Original_Column_2,Original_Column_3,Original_Column_4,Original_Column_5,Original_Column_6,Original_Column_7,Original_Column_8,Original_Column_9,Original_Column_10,Original_Column_11,Original_Column_12
0,stylegan3_image_expression,stylegan2_image_expression_3,our_image_expression_2,stylegan2_image_expression_0,real_image_expression_3,stylegan2_image_expression_2,real_image_expression_0,real_image_expression_1,real_image_expression_2,stylegan2_image_expression_1,our_image_expression_1,our_image_expression_3,our_image_expression_0
1,stylegan2_image_expression_2,real_image_expression_1,our_image_expression_0,our_image_expression_2,our_image_expression_3,real_image_expression_2,stylegan2_image_expression_1,our_image_expression_1,stylegan2_image_expression_3,stylegan2_image_expression_0,stylegan3_image_expression,real_image_expression_0,real_image_expression_3
2,our_image_expression_2,our_image_expression_3,stylegan2_image_expression_0,real_image_expression_2,stylegan2_image_expression_3,stylegan3_image_expression,real_image_expression_0,stylegan2_image_expression_1,our_image_expression_1,our_image_expression_0,real_image_expression_3,real_image_expression_1,stylegan2_image_expression_2
3,stylegan2_image_expression_1,our_image_expression_1,real_image_expression_2,stylegan2_image_expression_2,our_image_expression_0,stylegan3_image_expression,our_image_expression_2,our_image_expression_3,real_image_expression_1,real_image_expression_0,stylegan2_image_expression_0,real_image_expression_3,stylegan2_image_expression_3
4,real_image_expression_1,stylegan2_image_expression_0,stylegan2_image_expression_1,real_image_expression_3,real_image_expression_0,our_image_expression_1,our_image_expression_3,stylegan2_image_expression_2,our_image_expression_0,real_image_expression_2,stylegan2_image_expression_3,our_image_expression_2,stylegan3_image_expression
5,our_image_expression_3,stylegan2_image_expression_3,stylegan2_image_expression_1,real_image_expression_3,real_image_expression_0,real_image_expression_2,stylegan2_image_expression_0,our_image_expression_0,our_image_expression_2,stylegan3_image_expression,real_image_expression_1,our_image_expression_1,stylegan2_image_expression_2
6,our_image_expression_3,real_image_expression_2,real_image_expression_3,real_image_expression_1,our_image_expression_2,stylegan3_image_expression,real_image_expression_0,our_image_expression_0,our_image_expression_1,stylegan2_image_expression_0,stylegan2_image_expression_3,stylegan2_image_expression_1,stylegan2_image_expression_2
7,stylegan3_image_expression,real_image_expression_3,our_image_expression_1,real_image_expression_2,real_image_expression_0,our_image_expression_3,stylegan2_image_expression_0,stylegan2_image_expression_1,our_image_expression_2,real_image_expression_1,stylegan2_image_expression_2,our_image_expression_0,stylegan2_image_expression_3
8,our_image_expression_2,stylegan2_image_expression_3,real_image_expression_2,stylegan2_image_expression_0,stylegan2_image_expression_1,stylegan3_image_expression,real_image_expression_1,our_image_expression_1,our_image_expression_3,our_image_expression_0,real_image_expression_0,stylegan2_image_expression_2,real_image_expression_3
9,stylegan2_image_expression_3,stylegan2_image_expression_1,real_image_expression_0,our_image_expression_1,stylegan2_image_expression_2,our_image_expression_0,real_image_expression_3,our_image_expression_3,real_image_expression_2,stylegan2_image_expression_0,stylegan3_image_expression,real_image_expression_1,our_image_expression_2


In [21]:
with pd.ExcelWriter(out_path, engine='openpyxl') as writer:
    alignment_df.to_excel(writer, sheet_name='alignment', index=False)
    fidelity_df.to_excel(writer, sheet_name='fidelity', index=False)
    source_table.to_excel(writer, sheet_name='fidelity_source', index=True)
    shuffled_expression_table.to_excel(writer, sheet_name='fidelity_shuffled', index=True)